# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
from statsmodels.stats.multitest import multipletests
import torch
import pickle
import shutil
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
from src.pt.hyper_opt import train_hyper_opt
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from scipy import stats
import optuna
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import matplotlib.cm


def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

# Comparing multiple features sets

In [ ]:
path = f"D:/YandexDisk/Work/bbd/mriya"

feats_sets_models = {
    'Anthropometry': '127',
    'Complete Blood Count': '113',
    'Blood Biochemical': '125',
    'Electrocardiography': '70',
    'Echocardiography': '108',
    'Sphygmocardiography': '112',
}

feats_set_rename = {
    'Anthropometry': 'Антропометрия',
    'Complete Blood Count': 'Общий анализ крови',
    'Blood Biochemical': 'Биохимический анализ крови',
    'Electrocardiography': 'Электрокардиография',
    'Echocardiography': 'Эхокардиография',
    'Sphygmocardiography': 'Сфигмография',
    
}

colors = distinctipy.get_colors(len(feats_sets_models), [mcolors.hex2color(mcolors.CSS4_COLORS['black']), mcolors.hex2color(mcolors.CSS4_COLORS['white'])], rng=42)
colors_feats_sets = {x: colors[x_id] for x_id, x in enumerate(feats_sets_models.keys())}

## Age histograms

In [ ]:
n_rows = 2
n_cols = 3
fig_width = 15
fig_height = 9
hist_bins = np.linspace(5, 115, 23)

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={}, sharey=True, sharex=True)
for fs_id, (fs, model_id) in enumerate(feats_sets_models.items()):
    row_id, col_id = divmod(fs_id, n_cols)
    
    df_res = pd.read_excel(f"{path}/{fs}/models/DANet/{model_id}/df.xlsx", index_col=0)
    
    histplot = sns.histplot(
        data=df_res,
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        color=colors_feats_sets[fs],
        ax=axs[row_id, col_id]
    )
    axs[row_id, col_id].set(xlim=(0, 120))
    axs[row_id, col_id].set_title(f"{feats_set_rename[fs]} ({df_res.shape[0]})")
fig.tight_layout()    
fig.savefig(f"{path}/hist_age_for_feats.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/hist_age_for_feats.pdf", bbox_inches='tight')
plt.close(fig)

## Scatters and KDEs with metrics

In [ ]:
n_rows = 2 * 3
n_cols = 3
fig_width = 12
fig_height = 12

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), height_ratios=[0.2, 0.8, 0.1]*2, gridspec_kw={'wspace':0.35, 'hspace': 0.05}, sharey=False, sharex=False)

df_metrics_global = pd.DataFrame(index=list(feats_sets_models.keys()), columns=['MAE', 'Rho', 'Bias'])

for fs_id, (fs, model_id) in enumerate(feats_sets_models.items()):
    row_id, col_id = divmod(fs_id, n_cols)
    row_id_table = row_id * 3
    row_id_scatter = row_id * 3 + 1
    row_id_empty = row_id * 3 + 2
    
    fs_color = colors_feats_sets[fs]
    
    df_results = pd.read_excel(f"{path}/{fs}/models/DANet/{model_id}/df.xlsx", index_col=0)
    df_metrics = pd.read_excel(f"{path}/{fs}/models/DANet/{model_id}/metrics.xlsx", index_col=0)
    
    df_metrics_global.at[fs, 'MAE'] = df_metrics.at['Test', 'mean_absolute_error_unbiased']
    df_metrics_global.at[fs, 'Rho'] = df_metrics.at['Test', 'pearson_corrcoef_unbiased']
    df_metrics_global.at[fs, 'Bias'] = df_metrics.at['Test', 'bias_unbiased']
    
    xy_min = df_results[['Age', 'Prediction Unbiased']].min().min()
    xy_max = df_results[['Age', 'Prediction Unbiased']].max().max()
    xy_ptp = xy_max - xy_min

    df_table = pd.DataFrame(index=['MAE', fr"Pearson $\mathbf{{\rho}}$", "Bias"], columns=['Train', 'Validation', 'Test'])
    for part in ['Train', 'Validation', 'Test']:
        df_table.at['MAE', part] = f"{df_metrics.at[part, 'mean_absolute_error_unbiased']:0.3f}"
        df_table.at[fr"Pearson $\mathbf{{\rho}}$", part] = f"{df_metrics.at[part, 'pearson_corrcoef_unbiased']:0.3f}"
        df_table.at["Bias", part] = f"{df_metrics.at[part, 'bias_unbiased']:0.3f}"
    
    col_defs = [
        ColumnDefinition(
            name="index",
            title='',
            textprops={"ha": "center", "weight": "bold"},
            width=2.5,
            # border="both",
            group=feats_set_rename[fs],
        ),
        ColumnDefinition(
            name="Train",
            textprops={"ha": "left"},
            width=1.5,
            border="left",
            group=feats_set_rename[fs],
        ),
        ColumnDefinition(
            name="Validation",
            textprops={"ha": "left"},
            width=1.5,
            group=feats_set_rename[fs],
        ),
        ColumnDefinition(
            name="Test",
            textprops={"ha": "left"},
            width=1.5,
            group=feats_set_rename[fs],
        )
    ]

    table = Table(
        df_table,
        column_definitions=col_defs,
        row_dividers=True,
        footer_divider=False,
        ax=axs[row_id_table, col_id],
        textprops={"fontsize": 8},
        row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
        col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
        column_border_kw={"linewidth": 1, "linestyle": "-"},
    ).autoset_fontcolors(colnames=['Train', 'Validation', 'Test'])

    kdeplot = sns.kdeplot(
        data=df_results.loc[df_results['Group'] != 'Test', :],
        x='Age',
        y='Prediction Unbiased',
        fill=True,
        cbar=False,
        color=fs_color,
        thresh=0.05,
        cut=0,
        legend=False,
        ax=axs[row_id_scatter, col_id]
    )
    scatter = sns.scatterplot(
        data=df_results.loc[df_results['Group'] == 'Test', :],
        x='Age',
        y="Prediction Unbiased",
        linewidth=0.5,
        alpha=0.8,
        edgecolor="k",
        s=35,
        color=fs_color,
        ax=axs[row_id_scatter, col_id],
    )
    axs[row_id_scatter, col_id].axline((0, 0), slope=1, color="black", linestyle=":")
    axs[row_id_scatter, col_id].set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    axs[row_id_scatter, col_id].set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    
    axs[row_id_empty, col_id].axis('off')

fig.tight_layout()    
fig.savefig(f"{path}/models_for_feats.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/models_for_feats.pdf", bbox_inches='tight')
plt.close(fig)

df_metrics_global.rename(index=feats_set_rename, inplace=True)

col_defs = [
    ColumnDefinition(
        name="index",
        title="Features Set",
        textprops={"ha": "right", "weight": "bold"},
        width=2.25,
    ),
    ColumnDefinition(
        name="MAE",
        title="Test\nMAE",
        textprops={"ha": "center"},
        formatter="{:.3f}",
        cmap=normed_cmap(df_metrics_global["MAE"].dropna(), cmap=matplotlib.cm.Reds, num_stds=2.5),
        width=1.0,
    ),
    ColumnDefinition(
        name="Rho",
        title="Test\n" + r"Pearson $\rho$",
        textprops={"ha": "center"},
        formatter="{:.3f}",
        cmap=normed_cmap(df_metrics_global["Rho"].dropna(), cmap=matplotlib.cm.Greens, num_stds=2.5),
        width=1.0,
        border="left"
    ),
    ColumnDefinition(
        name="Bias",
        title="Test\nBias",
        textprops={"ha": "center"},
        formatter="{:.3f}",
        cmap=centered_cmap(df_metrics_global["Bias"].dropna(), cmap=matplotlib.cm.seismic, num_stds=2.5),
        width=1.0,
        border="left"
    ),
]
# df_metrics_global.insert(0, 'Feature Set', df_metrics_global.index)

fig, ax = plt.subplots()
table = Table(
    df_metrics_global,
    column_definitions=col_defs,
    row_dividers=True,
    footer_divider=False,
    odd_row_color="#ffffff",
    even_row_color="#f0f0f0",
    ax=ax,
    # textprops={"fontsize": 10},
    row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
    col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
    column_border_kw={"linewidth": 1, "linestyle": "-"},
).autoset_fontcolors(colnames=df_metrics_global.columns.to_list())
fig.savefig(f"{path}/test_metrics_for_feats.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/test_metrics_for_feats.pdf", bbox_inches='tight')
plt.close(fig)

## Features importance

In [ ]:
from sqlalchemy import column


n_rows = 1
n_cols = 6
fig_width = 20
fig_height = 9

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey=False, sharex=False)

for fs_id, (fs, model_id) in enumerate(feats_sets_models.items()):
    row_id, col_id = divmod(fs_id, n_cols)
    
    fs_color = colors_feats_sets[fs]
    
    fs_feats = pd.read_excel(f"{path}/{fs}/feats.xlsx", index_col=0)
    feats_cnt = fs_feats.index[fs_feats['data_type'].isin(['decimal', 'integer'])].to_list()
    
    df_results = pd.read_excel(f"{path}/{fs}/models/DANet/{model_id}/df.xlsx", index_col=0)
    df_metrics = pd.read_excel(f"{path}/{fs}/models/DANet/{model_id}/metrics.xlsx", index_col=0)
    df_explain = pd.read_excel(f"{path}/{fs}/models/DANet/{model_id}/explanation.xlsx", index_col=0)
    
    df_fi = pd.DataFrame(index=feats_cnt, columns=['abs(|SHAP|)'])
    for f in feats_cnt:
        df_fi.at[f, 'abs(|SHAP|)'] = df_explain[f].abs().mean()
    df_fi.sort_values(['abs(|SHAP|)'], ascending=[False], inplace=True)
    
    df_fi = df_fi.head(30)
    df_fi['Features'] = df_fi.index.values
    barplot = sns.barplot(
        data=df_fi,
        x='abs(|SHAP|)',
        y='Features',
        color=fs_color,
        edgecolor='black',
        dodge=False,
        ax=axs[col_id]
    )
    for container in barplot.containers:
        barplot.bar_label(container, label_type='edge', fmt='%0.2f', fontsize=12, padding=4.0)
    axs[col_id].set_title(feats_set_rename[fs], fontsize='large')
    axs[col_id].set_ylabel('')

fig.tight_layout()    
fig.savefig(f"{path}/barplot_importance_for_feats.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/barplot_importance_for_feats.pdf", bbox_inches='tight')
plt.close(fig)

# Features correlations

In [ ]:
feats_set_rename = {
    'Anthropometry': 'Антропометрия',
    'Complete Blood Count': 'Общий анализ крови',
    'Blood Biochemical': 'Биохимический анализ крови',
    'Electrocardiography': 'Электрокардиография',
    'Echocardiography': 'Эхокардиография',
    'Sphygmocardiography': 'Сфигмография',
    'Arterial Stiffness': 'Cосудистая жесткость',
    'Best Correlation': 'Лучшие из разных групп'
}


for fs in feats_set_rename:
    fs_feats = pd.read_excel(f"{path}/{fs}/feats.xlsx", index_col=0)
    fs_df = pd.read_excel(f"{path}/{fs}/data.xlsx", index_col=0)
    
    feats_cnt = ['Age'] + fs_feats.index[fs_feats['data_type'].isin(['decimal', 'integer'])].to_list()
    df_corr = pd.DataFrame(data=np.zeros(shape=(len(feats_cnt), len(feats_cnt))), index=feats_cnt, columns=feats_cnt)
    for f_id_1 in range(len(feats_cnt)):
        for f_id_2 in range(f_id_1, len(feats_cnt)):
            f_1 = feats_cnt[f_id_1]
            f_2 = feats_cnt[f_id_2]
            if f_id_1 != f_id_2:
                vals_1 = fs_df.loc[:, f_1].values
                vals_2 = fs_df.loc[:, f_2].values
                corr, pval = stats.pearsonr(vals_1, vals_2)
                df_corr.at[f_2, f_1] = pval
                df_corr.at[f_1, f_2] = corr
            else:
                df_corr.at[f_2, f_1] = np.nan
    selection = np.tri(df_corr.shape[0], df_corr.shape[1], -1, dtype=bool)
    df_fdr = df_corr.where(selection).stack().reset_index()
    df_fdr.columns = ['row', 'col', 'pval']
    _, df_fdr['pval_fdr_bh'], _, _ = multipletests(df_fdr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
    nzmin = df_fdr['pval_fdr_bh'][df_fdr['pval_fdr_bh'].gt(0)].min(0) * 0.5
    df_fdr['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
    df_corr_fdr = df_corr.copy()
    for line_id in range(df_fdr.shape[0]):
        df_corr_fdr.loc[df_fdr.at[line_id, 'row'], df_fdr.at[line_id, 'col']] = -np.log10(df_fdr.at[line_id, 'pval_fdr_bh'])
    df_corr_fdr.to_excel(f"{path}/{fs}/feats_pearsonr.xlsx")
        
    sns.set_theme(style='ticks')
    # fig, ax = plt.subplots(figsize=(4.5 + 0.25 * len(feats_cnt), 2.5 + 0.25 * len(feats_cnt)))
    fig, ax = plt.subplots(figsize=(4.5 + 0.25 * len(feats_cnt), 2.5 + 0.25 * len(feats_cnt)))
    cmap_triu = plt.get_cmap("seismic").copy()
    mask_triu=np.tri(len(feats_cnt), len(feats_cnt), -1, dtype=bool)
    heatmap_diff = sns.heatmap(
        df_corr_fdr,
        mask=mask_triu,
        annot=True,
        fmt=".2f",
        center=0.0,
        cmap=cmap_triu,
        linewidth=0.1,
        linecolor='black',
        annot_kws={"fontsize": 25 / np.sqrt(len(df_corr_fdr.values))},
        ax=ax
    )
    ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$", size=13)
    for spine in ax.figure.axes[-1].spines.values():
        spine.set(visible=True, lw=0.25, edgecolor="black")
        
    cmap_tril = plt.get_cmap("viridis").copy()
    cmap_tril.set_under('black')
    mask_tril=np.tri(len(feats_cnt), len(feats_cnt), -1, dtype=bool).T
    heatmap_pval = sns.heatmap(
        df_corr_fdr,
        mask=mask_tril,
        annot=True,
        fmt=".1f",
        vmin=-np.log10(0.05),
        cmap=cmap_tril,
        linewidth=0.1,
        linecolor='black',
        annot_kws={"fontsize": 25 / np.sqrt(len(df_corr_fdr.values))},
        ax=ax
    )
    ax.figure.axes[-1].set_ylabel(r"$-\log_{10}(\mathrm{p-value})$", size=13)
    for spine in ax.figure.axes[-1].spines.values():
        spine.set(visible=True, lw=0.25, edgecolor="black")
    ax.set_xlabel('', fontsize=16)
    ax.set_ylabel('', fontsize=16)
    ax.set_title(feats_set_rename[fs])
    plt.savefig(f"{path}/{fs}/feats_pearsonr.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/{fs}/feats_pearsonr.pdf", bbox_inches='tight')
    plt.close(fig)

# Models with selected features

In [ ]:
feats_sets_models = {
    'Best Correlation': '45',
    'Arterial Stiffness': '86',
}

feats_set_rename = {
    'Arterial Stiffness': 'Cосудистая жесткость',
    'Best Correlation': 'Лучшие из разных групп'
}

colors_feats_sets = {
    'Best Correlation': 'crimson',
    'Arterial Stiffness': 'dodgerblue',
}

for fs_id, (fs, model_id) in enumerate(feats_sets_models.items()):
    
    fs_feats = pd.read_excel(f"{path}/{fs}/feats.xlsx", index_col=0)
    fs_results = pd.read_excel(f"{path}/{fs}/models/DANet/{model_id}/df.xlsx", index_col=0)
    fs_metrics = pd.read_excel(f"{path}/{fs}/models/DANet/{model_id}/metrics.xlsx", index_col=0)
    df_explain = pd.read_excel(f"{path}/{fs}/models/DANet/{model_id}/explanation.xlsx", index_col=0)
    
    xy_min = fs_results[['Age', 'Prediction Unbiased']].min().min()
    xy_max = fs_results[['Age', 'Prediction Unbiased']].max().max()
    xy_ptp = xy_max - xy_min
    
    feats_cnt = fs_feats.index[fs_feats['data_type'].isin(['decimal', 'integer'])].to_list()
    
    n_rows = 4
    n_cols = 1
    fig_height = 6
    fig_width = 4
    sns.set_theme(style='ticks')
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), height_ratios=[0.2, 0.2, 0.8, 0.15], gridspec_kw={'wspace':0.25, 'hspace': 0.05}, sharey=False, sharex=False)

    row_id_table = 0
    row_id_hist = 1
    row_id_scatter = 2
    row_id_empty = 3

    df_table = pd.DataFrame(index=['MAE', fr"Pearson $\mathbf{{\rho}}$", "Bias"], columns=['Train', 'Validation', 'Test'])
    for part in ['Train', 'Validation', 'Test']:
        df_table.at['MAE', part] = f"{fs_metrics.at[part, 'mean_absolute_error_unbiased']:0.3f}"
        df_table.at[fr"Pearson $\mathbf{{\rho}}$", part] = f"{fs_metrics.at[part, 'pearson_corrcoef_unbiased']:0.3f}"
        df_table.at["Bias", part] = f"{fs_metrics.at[part, 'bias_unbiased']:0.3f}"

    col_defs = [
        ColumnDefinition(
            name="index",
            title='',
            textprops={"ha": "center", "weight": "bold"},
            width=2.5,
            group=feats_set_rename[fs],
        ),
        ColumnDefinition(
            name="Train",
            textprops={"ha": "left"},
            width=1.5,
            border="left",
            group=feats_set_rename[fs],
        ),
        ColumnDefinition(
            name="Validation",
            textprops={"ha": "left"},
            width=2.2,
            group=feats_set_rename[fs],
        ),
        ColumnDefinition(
            name="Test",
            textprops={"ha": "left"},
            width=1.5,
            group=feats_set_rename[fs],
        )
    ]

    table = Table(
        df_table,
        column_definitions=col_defs,
        row_dividers=True,
        footer_divider=False,
        ax=axs[row_id_table],
        textprops={"fontsize": 8},
        row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
        col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
        column_border_kw={"linewidth": 1, "linestyle": "-"},
    ).autoset_fontcolors(colnames=['Train', 'Validation', 'Test'])

    hist_bins = np.linspace(0, 120, 25)
    histplot = sns.histplot(
        data=fs_results,
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        color=colors_feats_sets[fs],
        ax=axs[row_id_hist]
    )
    axs[row_id_hist].set_xticks([])
    axs[row_id_hist].set_xlim(xy_min-0.05*xy_ptp, xy_max+0.05*xy_ptp)
    axs[row_id_hist].set_ylabel("Count")

    kdeplot = sns.kdeplot(
        data=fs_results.loc[fs_results['Group'].isin(['Train', 'Validation']), :],
        x='Age',
        y='Prediction Unbiased',
        fill=True,
        cbar=False,
        thresh=0.05,
        color=colors_feats_sets[fs],
        cut=0,
        legend=False,
        ax=axs[row_id_scatter]
    )
    scatter = sns.scatterplot(
        data=fs_results.loc[fs_results['Group'] == 'Test', :],
        x='Age',
        y="Prediction",
        linewidth=0.5,
        alpha=0.8,
        edgecolor="k",
        s=15,
        color=colors_feats_sets[fs],
        ax=axs[row_id_scatter],
    )
    axs[row_id_scatter].axline((0, 0), slope=1, color="black", linestyle=":")
    axs[row_id_scatter].set_xlim(xy_min-0.05*xy_ptp, xy_max+0.05*xy_ptp)
    axs[row_id_scatter].set_ylim(xy_min-0.05*xy_ptp, xy_max+0.05*xy_ptp)
    axs[row_id_scatter].set_ylabel("Prediction Unbiased")
    axs[row_id_scatter].set_xlabel("Age")
    axs[row_id_empty].axis('off')
    fig.tight_layout()
    fig.savefig(f"{path}/{fs}/model.png", bbox_inches='tight', dpi=400)
    fig.savefig(f"{path}/{fs}/model.pdf", bbox_inches='tight')
    plt.close(fig)
    
    df_fi = pd.DataFrame(index=feats_cnt, columns=['abs(|SHAP|)'])
    for f in feats_cnt:
        df_fi.at[f, 'abs(|SHAP|)'] = df_explain[f].abs().mean()
    df_fi.sort_values(['abs(|SHAP|)'], ascending=[False], inplace=True)
    
    fig, ax = plt.subplots(figsize=(8, 8))
    df_fi['Features'] = df_fi.index.values
    barplot = sns.barplot(
        data=df_fi,
        x='abs(|SHAP|)',
        y='Features',
        color=colors_feats_sets[fs],
        edgecolor='black',
        dodge=False,
        ax=ax
    )
    for container in barplot.containers:
        barplot.bar_label(container, label_type='edge', fmt='%0.2f', fontsize=12, padding=4.0)
    ax.set_title(feats_set_rename[fs], fontsize='large')
    ax.set_ylabel('')
    fig.tight_layout()    
    fig.savefig(f"{path}/{fs}/importance_for_feats.png", bbox_inches='tight', dpi=300)
    fig.savefig(f"{path}/{fs}/importance_for_feats.pdf", bbox_inches='tight')
    plt.close(fig)
    